In [ ]:
# this is the mutual information regression that uses KNN - uses preassigned number of features

from sklearn.feature_selection import SelectKBest
from functools import partial
from sklearn.feature_selection import mutual_info_regression
 
score_func = partial(mutual_info_regression, discrete_features=[0], random_state=68)
 
selection = SelectKBest(score_func=score_func, k=10)
#more features more better
 
selection.fit_transform(X, y)

print(X[X.columns[selection.get_support(indices=True)]])

In [ ]:
#uses a random forest regression in order to determine importances for each category by using gini coefficient

from sklearn.ensemble import RandomForestRegressor

modelRF = RandomForestRegressor(n_estimators=100)
modelRF.fit(X,y)
importances = modelRF.feature_importances_


finaldf = pd.DataFrame({"Features":pd.DataFrame(X).columns,"Importances":importances})
finaldf.set_index("Importances")
finaldf = finaldf.sort_values("Importances")
finaldf.plot.bar(color = "teal")

In [ ]:
#this uses an exhaustive feature selection to test all possible combinations of the features (it takes a long time) 

#number of combinations is summation from min features to max features of (maxfeaturesCn)

RF = RandomForestRegressor()

efs = EFS(RF, min_features = 1, max_features = 15, scoring = "neg_mean_squared_error", print_progress = True,cv=5,n_jobs=-1)

efs.fit(X,y)

print("Best MSE Score: %.2f"% efs.best_score_ *(-1))
print("Best subset:", efs.best_idx_)
selected_features = X.columns[list(efs.best_idx_)]
print(selected_features)

print(selected_features)

In [ ]:
#Sequential feature selection using Support vector regression - uses preassigned number of features

#SVR
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt

X = df.loc[:,'calib':'pLowEffortLowRewardPressRate']

regress = SVR(kernel = 'rbf', gamma = 0.05)
sffsf = SFS(regress, k_features=8, forward=True,floating=False, verbose=0,scoring=None, cv=2, n_jobs=1,).fit(X, y)
print(sffsf.k_feature_names_)


plot_sfs(sffsf.get_metric_dict())
plt.show()

#sequential feature selection using random forest regressor 
from sklearn.ensemble import RandomForestRegressor
regress = RandomForestRegressor(n_estimators = 30, random_state = 0)
X = df.loc[:,'calib':'pLowEffortLowRewardPressRate']
sffsf = SFS(regress, k_features=50, forward=True,floating=False, verbose=0,scoring=None, cv=2, n_jobs=1,).fit(X, y)
print(sffsf.k_feature_names_)

plot_sfs(sffsf.get_metric_dict())
plt.show()

#sequential feature selection using a sklearn ANN
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import RidgeCV
from sympy.combinatorics.subsets import Subset
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

X = df.loc[:,'calib':'pLowEffortLowRewardPressRate']

reg = MLPRegressor(hidden_layer_sizes=(2000,5), activation='relu', solver='adam', max_iter=5000)

sffsf = SFS(clf, k_features=5, forward=True,floating=True, verbose=0,scoring=None, cv=2, n_jobs=1,).fit(X, y)
print(sffsf.k_feature_names_)

plot_sfs(sffsf.get_metric_dict())
plt.show()


#SFS can be used with any machine learning method (logistic regression, SVR, ANN, etc)

In [ ]:
#permutation test


p = 50000
permute_df = pd.DataFrame(columns=['permuted_avg_mse'])

#names dataframe

for i in range (p):
    df1['rateExerciseCompletion'] = df1['rateExerciseCompletion'].sample(n=19).values
    X = df1.loc[:,['ctRmHighEffortHighReward','ctRmMedEffort','ctRmLowEffortMedReward','LowEffortLowRewardPts','ctRmMedReward','ctRmMedEffortMedReward','HighEffortLowRewardTime']]
    y = df1.loc[:,['rateExerciseCompletion']]
    loss1, avg_mse1 = model.evaluate(X,y, batch_size=16,verbose=1)
    permute_df.loc[i, 'permuted_avg_mse'] = avg_mse1
    
    
#evaluates mse for P permutations using the model and saves them to dataframe 

count = 0
for i in range (p):
    if permute_df.iloc[i,0]<=avg_mse:
        count+=1
p_value = ((count+1)/(p+1))
print("P_value for overall average MSE:", p_value)
print("Count for overall average MSE:",count)



#count is number of times permutation mse is lower than avg_mse for model, p_value is ratio of count+1 over P+1